In [ ]:
import pandas as pd 
import geopandas
from shapely.geometry import Point
import requests

In [ ]:
## We should have 34,753 LSOAs and 7,201 in total
## Fro https://www.ons.gov.uk/methodology/geography/ukgeographies/censusgeography

In [ ]:
import tempfile
from urllib.request import urlretrieve
import zipfile
import os

# We're going to use temporary files to download and unzip the ONS data.  This just means these temp files will be deleted for us at the end of the session
tempdir = tempfile.TemporaryDirectory()
tempfile = tempfile.NamedTemporaryFile()

# http://geoportal.statistics.gov.uk/datasets/b7c49538f0464f748dd7137247bbc41c_0

# http://geoportal1-ons.opendata.arcgis.com/datasets/b7c49538f0464f748dd7137247bbc41c_0
url = "https://opendata.arcgis.com/datasets/b7c49538f0464f748dd7137247bbc41c_0.zip"
zipped_shp = urlretrieve(url, tempfile.name)

with zipfile.ZipFile(zipped_shp[0],"r") as zip_ref:
    zip_ref.extractall(tempdir.name)

shapefile_name = [f for f in os.listdir(tempdir.name) if f.endswith("shp")][0]
shapefile_path = os.path.join(tempdir.name, shapefile_name)

lsoas = geopandas.read_file(shapefile_path)
print(len(lsoas))

In [ ]:
lsoas["lng"] = lsoas["geometry"].x 
lsoas["lat"] = lsoas["geometry"].y

In [ ]:
# http://geoportal.statistics.gov.uk/datasets/output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-december-2011-lookup-in-england-and-wales
# http://geoportal.statistics.gov.uk/datasets/output-area-to-lsoa-to-msoa-to-local-authority-district-december-2017-lookup-with-area-classifications-in-great-britain
lookup_url = "https://opendata.arcgis.com/datasets/6ecda95a83304543bc8feedbd1a58303_0.csv"
lookup = pd.read_csv(lookup_url)
lookup.drop("OA11CD", axis=1, inplace=True)
lookup = lookup.drop_duplicates("LSOA11CD")

In [ ]:
merged = lsoas.merge(lookup, left_on = "lsoa11cd", right_on = "LSOA11CD" )
len(merged)

In [ ]:
lookup.head()

In [ ]:
msoas = merged["MSOA11CD"].unique()
lsoas = merged["lsoa11cd"].unique()

In [ ]:
# Partition by MSOA but one file per LSOA

In [ ]:
https://osrm-great-britain.apps.alpha.mojanalytics.xyz/route/v1/driving/-3.9528905,51.6197292;-1.2960193,51.6594136

In [ ]:
base_url = "https://osrm-great-britain.apps.alpha.mojanalytics.xyz/route/v1/driving/{from_lng},{from_lat};{to_lng},{to_lat}"

In [ ]:
lsoa = lsoas[0]
lsoa

In [ ]:
from_lsoa = lsoa
from_lng = merged[merged["lsoa11cd"] == lsoa].lng[0]
from_lat = merged[merged["lsoa11cd"] == lsoa].lat[0]

In [ ]:
import json
def get_results(url, row, from_lsoa):
    return_row = {}
    r = requests.get(url)
    routes = json.loads(r.text)
    return_row["distance"] = routes["routes"][0]["distance"]
    return_row["duration"] = routes["routes"][0]["duration"]
    return_row["from_lsoa11cd"] = from_lsoa
    return_row["to_lsoa11cd"] = row["lsoa11cd"]
    return return_row
    
    

In [ ]:
%%time
new_df = []
counter = 0
for r in merged[:1000].iterrows():

    if counter % 100 == 0:
        print(f"Done {counter} rows")
    counter += 1
    
    row = r[1]
    to_lat = row["lat"]
    to_lng = row["lng"]
    url = f"https://www.google.co.uk/maps/dir/{from_lat},{from_lng}/{to_lat},{to_lng}"
    url = f"https://osrm-great-britain.apps.alpha.mojanalytics.xyz/route/v1/driving/{from_lng},{from_lat};{to_lng},{to_lat}"
    return_row = get_results(url, row, from_lsoa)
    new_df.append(return_row)
print("done")

In [ ]:
sec = 41 * (34000**2) / 1000

In [ ]:
sec/(60*60*24)

In [ ]:
"{:,.0f}".format(34753 ** 2)

In [ ]:
len(lookup["MSOA11CD"].unique())

In [ ]:
7201 ** 2